In [ ]:
# Step 1: Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
import ccxt

In [ ]:
# Connecting to Binance and Fetching 5 min OHLCV
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '5m'
limit = 1000  # max 1500 per request
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df.set_index('Timestamp', inplace=True)

# Calculating RSI 
rsi = RSIIndicator(close=df["Close"], window=14)
df["RSI"] = rsi.rsi()

# preview the result
print(df[["Close", "RSI"]].tail(10))

In [ ]:
# Step 4: Building an RSI Strat x >70 sell, x < 30buy Simple
## This must include the log and simulates trade 
in_position = False  # Track if I'm currently in a trade
trades = []          # Store all completed trades

for i in range(1, len(df)):
    rsi = df["RSI"].iloc[i]
    price = df["Close"].iloc[i]
    time = df.index[i]

    if not in_position and rsi < 30:
        # Buy signal
        entry_price = price
        entry_time = time
        in_position = True

    elif in_position and rsi > 70:
        # Sell signal
        exit_price = price
        exit_time = time
        pnl = (exit_price - entry_price) / entry_price  # % gain/loss
        trades.append({
            "Entry Time": entry_time,
            "Exit Time": exit_time,
            "Entry Price": entry_price,
            "Exit Price": exit_price,
            "PnL %": pnl * 100
        })
        in_position = False


In [ ]:
trades_df = pd.DataFrame(trades)
print(trades_df)

In [ ]:
df["Returns"] = df["Close"].pct_change()
df["Position"] = 0

for i in range(len(trades)):
    entry = trades[i]["Entry Time"]
    exit = trades[i]["Exit Time"]
    df.loc[entry:exit, "Position"] = 1

df["Strategy_Returns"] = df["Returns"] * df["Position"]

In [ ]:
(1 + df[["Returns", "Strategy_Returns"]]).cumprod().plot(figsize=(14, 6))
plt.title("RSI Strategy vs Buy & Hold (BTC/USDT 15m)")
plt.xlabel("Time")
plt.ylabel("Growth")
plt.grid()
plt.show()

In [ ]:
# Convert PnL % to growth factor (e.g., 2.5% → 1.025)
trades_df["Growth"] = 1 + (trades_df["PnL %"] / 100)

# Calculate cumulative equity growth
trades_df["Equity Curve"] = trades_df["Growth"].cumprod()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(trades_df["Exit Time"], trades_df["Equity Curve"], marker='o')
plt.title("RSI Strategy: Cumulative PnL")
plt.xlabel("Exit Time")
plt.ylabel("Account Growth (x)")
plt.grid(True)
plt.show()